In [ ]:
### 画像処理用
import cv2
import numpy as np
import math

### OCR処理用
import easyocr

### jupyter表示用
import matplotlib.pyplot as plt
import copy
import json

In [ ]:
input_file_path = "input/sample_img2.jpg"

img = cv2.imread(input_file_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_tmp = copy.deepcopy(img)
plt.imshow(img)
plt.show()

In [ ]:
### 大津の手法による二値化処理
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret,bin_img = cv2.threshold(gray_img, 10, 255, cv2.THRESH_OTSU)
plt.imshow(cv2.cvtColor(bin_img, cv2.COLOR_GRAY2BGR))
plt.show()

In [ ]:
### 規定以上の大きい領域を抽出する関数
def findLargeArea(bin_img, th_area: int = 10000):
    # 輪郭抽出
    contours, hierarchy = cv2.findContours(bin_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # 面積の大きいもののみ選別
    large_areas = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > th_area:
            epsilon = 0.0001*cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt,epsilon, True)
            large_areas.append([area, approx])
    large_areas = sorted(large_areas, reverse=True, key=lambda data: data[0])
    return [ area[1] for area in large_areas]

### 最大領域の抽出
### 規定以上の大きい領域を抽出する関数を利用（findLargeArea）
def findMaxArea(bin_img, th_area: int = 10000):
    return findLargeArea(bin_img, th_area)[0]


max_areas = findMaxArea(bin_img)
cv2.drawContours(img_tmp,max_areas,-1,(0,255,0),3)

print(len(max_areas))
plt.imshow(img_tmp)
plt.show()

In [ ]:
### 重心を算出する関数
def calCenter(area):
    mu = cv2.moments(area)
    x  = int(mu["m10"] / mu["m00"])
    y  = int(mu["m01"] / mu["m00"])
    return x, y

x, y = calCenter(max_areas)
print(x, y)

cv2.circle(img_tmp, (x,y), 4, 100, 2, 4)

plt.imshow(img_tmp)
plt.show()


In [ ]:
### 重心から遠い座標を中心に、近似する座標を除外する処理
### [out] max_point：dist_listの中で一番重心から遠い座標
def deleteNearPoint(dist_list, th_dist):
    dist_list   = sorted(dist_list, reverse=True, key=lambda data: data[0])
    _,far_point = dist_list[0]
    for n in reversed(range(len(dist_list))):
        cal_dist = np.linalg.norm(dist_list[n][1] - far_point)
        if th_dist > cal_dist:
            dist_list.pop(n)
            # print(len(dist_list))
    return far_point, dist_list

### 重心から遠い4座標を決定する関数
def detectPoint(area, x, y, th_coe: float = 0.75):
    out_points = []

    ### 重心からの距離を算出
    result_list = []
    center = np.array([x, y])
    for point in area:
        result_list.append([np.linalg.norm(center - point), point])

    ### 重心から一番遠い座標に近似する座標と判定する閾値の算出
    sort_list  = sorted(result_list, reverse=True, key=lambda data: data[0])
    max_dist,_ = sort_list[0]
    th_dist    = max_dist * th_coe
    while len(sort_list):
        far_point, sort_list = deleteNearPoint(sort_list, th_dist)
        out_points.append(far_point)
        # print(len(out_points))
    return out_points[:4]

c_points = detectPoint(max_areas, x, y)
print(c_points)

In [ ]:
for area in c_points:
    cv2.circle(img_tmp, (area[0]), 4, 100, 20, 4)
plt.imshow(img_tmp)
plt.show()

In [ ]:
def warpPerspective(in_rgb_img, points):
    try:

        # 変換前4点の座標　p1:左上　p2:左下 p3:右下 p4:右上
        points = sorted(points, key=lambda data: data[0][1])
        upside_point = sorted(points[:2], key=lambda data: data[0][0])
        downside_point = sorted(points[2:], key=lambda data: data[0][0])
        
        p1 = upside_point[0][0]
        p2 = downside_point[0][0]
        p3 = downside_point[1][0]
        p4 = upside_point[1][0]

        print("P1:", p1)
        print("P2:", p2)
        print("P3:", p3)
        print("P4:", p4)

        #　幅取得
        o_width_max = max(np.linalg.norm(p4 - p1), np.linalg.norm(p2 - p3))
        o_width_min = min(np.linalg.norm(p4 - p1), np.linalg.norm(p2 - p3))
        # 比率調整
        ratio    = o_width_max / o_width_min
        o_width  = math.floor(o_width_min * ratio)

        #　高さ取得
        o_height = max(np.linalg.norm(p2 - p1), np.linalg.norm(p4 - p3))
        o_height = math.floor(o_height * ratio)
        
        print("length_w:{}   length_h:{}".format(o_width, o_height))

        # 変換前の4点
        src = np.float32([p1, p2, p3, p4])

        # 変換後の4点
        dst = np.float32([[0, 0],[0, o_height],[o_width, o_height],[o_width, 0]])

        # 変換行列
        M = cv2.getPerspectiveTransform(src, dst)

        # 射影変換・透視変換する
        output = cv2.warpPerspective(in_rgb_img, M,(o_width, o_height))
    except Exception as e:
        print("error:{}".format(e.args))
        return None

    return output
daikei = warpPerspective(img, c_points)

plt.imshow(daikei)
plt.show()

In [ ]:
reader = easyocr.Reader(['ja', 'en'])#日本語：ja, 英語：en

results = reader.readtext(daikei)
for result in results:
    print("text:", result[1], result[0])

In [ ]:
def formatOCRpoints2Json(points):
    output_points = copy.deepcopy(points)
    for point in output_points:
        point[0] = int(point[0])
        point[1] = int(point[1])
    return output_points

def makeOCRresult2Json(ocr_results):
    output_list = []
    for oce_result in ocr_results:
        output               = {}
        output["Text"]       = oce_result[1]
        output["Points"]     = formatOCRpoints2Json(oce_result[0])
        output["Confidence"] = oce_result[2]
        output_list.append(output)
    return output_list

result_json = {}
result_json["File"] = input_file_path
result_json["Result"] = makeOCRresult2Json(results)
with open('search_result.json', 'w') as f:
    json.dump(result_json, f, indent=4, ensure_ascii=False)

In [ ]:
def drawBoxImage(img, points):
    print("  dwar points:{}, {}, {}, {}".\
        format(tuple(points[0]), tuple(points[1]),\
               tuple(points[2]), tuple(points[3])))

    return cv2.polylines(img, [points], True, (255, 0, 0), thickness = 2)

def drawOCRResultImage(img, ocr_results):
    for oce_result in ocr_results:
        print(oce_result[1])
        ### OCRの座標情報はList型のため、numpyに変換する
        points = np.array(oce_result[0],dtype=np.int32)
        drawBoxImage(img, points)

drawOCRResultImage(daikei, results)
plt.imshow(daikei)
plt.show()